In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("omitted_vscode_TD_dataset.csv" , index_col = 0)
df = df[df['text'].notna()]
df = df.drop_duplicates(subset=["text"], keep="last")
df = df.reset_index()
df.drop(columns=["index"], inplace=True)
df.head()

,text,label
0,wallet actor has a connection to electrum via...,1
1,end of the info no custom variables for ke...,1
2,replace hardcoded asset lists in various places,1
3,in s recent analysis of our ensembling algori...,1
4,as a service provide\r i need to deploy the se...,1


In [3]:
# Dataset to answer RQ - How well do LSTMs generalize to classify TD in new projects and datasets?
df_jira=pd.read_csv("jira_TD_dataset.csv" , index_col = 0)
df_jira = df_jira[df_jira['text'].notna()]
df_jira = df_jira.drop_duplicates(subset=["text"], keep="last")
df_jira = df_jira.reset_index()
df_jira.drop(columns=["index"], inplace=True)
df_jira.head()

,text,label
0,this is in kahaimplindexindexmanagerjavathe in...,0
1,from execution of there are additional schedu...,1
2,the incrbackup test does not appear to test ei...,1
3,uniquekeys is a kind of constraint its unreaso...,1
4,\r\rthere is no any artifacts,1


In [4]:
###Drop Nan Values
df=df.dropna()
df_jira = df_jira.dropna()

In [5]:
## Get the Independent Features
X=df.drop('label',axis=1)
X_jira = df_jira.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']
y_jira = df_jira['label']

In [7]:
X.shape

(119845, 1)

In [8]:
y.shape

(119845,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.12.0-rc1'

In [11]:
from keras.layers import Embedding
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Dropout

In [12]:
### Vocabulary size
voc_size=15000

### Onehot Representation

In [13]:
messages=X.copy()
messages_jira = X_jira.copy()

In [14]:
messages['text'][1]

'end of the   info   no custom variables for kernel as interpreter is not conda but is   \x1b\x1b codewindow failed to load workbench window   exit code      end standard tests with errors  exiting with test  killing vsc'

In [15]:
messages_jira['text'][1]

'from execution of  there are additional scheduled tasks which as of  do not support cancellation for the same reasons as the aforementioned jira these tasks should be allowed to be canceled where possible\xa0\r\r\xa0\r  compactblobstoretask\r  dockeruploadpurgetask\r  purgeapikeystask\r  rebuildassetuploadmetadatatask completed previously\r  rebuildbrowsenodestask completed previously\r  rebuildindextask\r  yumcreaterepotask not addressing at this time\r  repairyumproxycomponentstask class no longer exists'

In [16]:
messages.reset_index(inplace=True)
messages_jira.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\durga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
### Dataset Preprocessing for va_gov
ps_jira = PorterStemmer()
corpus_jira = []
for i in range(0, len(messages_jira)):
    review_jira = re.sub('[^a-zA-Z]', ' ', messages_jira['text'][i])
    review_jira = review_jira.lower()
    review_jira = review_jira.split()
    
    review_jira = [ps_jira.stem(word) for word in review_jira if not word in stopwords.words('english')]
    review_jira = ' '.join(review_jira)
    corpus_jira.append(review_jira)

In [21]:
corpus

['wallet actor connect electrum via bdk connect fail restart wallet supervisor actordesign need follow raii pattern outlin fix',
 'end info custom variabl kernel interpret conda codewindow fail load workbench window exit code end standard test error exit test kill vsc',
 'replac hardcod asset list variou place',
 'recent analysi ensembl algorithm come light spend lot time copi data ive notic throughout code lot copi datafram think understand copi whether theyr necessari impedi perform accept criteria issu benchmark profil unit test suit visual snakeviz could also link copypast result show earlier today would help highlight problem',
 'servic provid need deploy servic cloud user access anywher detail assumpt use ibm cloud servic provid uniqu url counter accept criteria gherkin given url counter user click link counter servic access',
 'servic provid need deploy servic cloud user access anywher detail assumpt use ibm cloud servic provid uniqu url counter accept criteria gherkin given url

In [22]:
corpus_jira

['kahaimplindexindexmanagerjavath index current maintain inmemori free list entri cost least byte ram issu free space file ram use oddli enoughi worri case suppos produc enqueu million messag stop suppos consum dequeu messag produc start time free entri reclaim seem like could use huge amount memori ye mb much suppos bunch consum henc consumermessageref one mayb im miss somethingperhap link list could push kaha store rewrit indexitem disk point previousnext free item thu maintain disk place perf reason mayb supplement cach might realli necessari os cach help outa lot place kaha listcontainerimpljava cach limit say cach item free list seem obviat stuff due essenti unboundedani thought',
 'execut addit schedul task support cancel reason aforement jira task allow cancel possibl compactblobstoretask dockeruploadpurgetask purgeapikeystask rebuildassetuploadmetadatatask complet previous rebuildbrowsenodestask complet previous rebuildindextask yumcreaterepotask address time repairyumproxycomp

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr_jira=[one_hot(words,voc_size)for words in corpus_jira] 

### Embedding Representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs_jira=pad_sequences(onehot_repr_jira,padding='pre',maxlen=sent_length)

In [25]:
embedded_docs[0]

array([    0,     0, 11239, 14080, 10117,  7026, 11950,  7018, 10117,
       12023,  1171, 11239,   145,  7717,  3229,  3709,  8482,  7159,
       14688,  9797])

In [26]:
embedded_docs_jira[0]

array([12227,  4147,  3459, 14602,  9268, 13301,  2433,  9921,  5443,
        2433,  7759,  3251, 14714,  2273,  5207, 13387, 14683,  2652,
        4628, 13201])

In [27]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            600000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 713,001
Trainable params: 713,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(119845, (119845,))

In [29]:
len(embedded_docs_jira),y_jira.shape

(828, (828,))

In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final_jira=np.array(embedded_docs_jira)
y_final_jira=np.array(y_jira)

In [31]:
X_final.shape,y_final.shape

((119845, 20), (119845,))

In [32]:
X_final_jira.shape, y_final_jira.shape

((828, 20), (828,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#JIRA dataset is entirely used to test on a model trained using VScode dataset

### Model Training

In [34]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


1255/1255 [==============================] - 37s 27ms/step - loss: 0.4613 - accuracy: 0.7817 - val_loss: 0.4349 - val_accuracy: 0.7974
Epoch 2/10
1255/1255 [==============================] - 36s 29ms/step - loss: 0.3842 - accuracy: 0.8285 - val_loss: 0.4482 - val_accuracy: 0.7953
Epoch 3/10
1255/1255 [==============================] - 38s 30ms/step - loss: 0.3314 - accuracy: 0.8560 - val_loss: 0.4559 - val_accuracy: 0.7889
Epoch 4/10
1255/1255 [==============================] - 35s 28ms/step - loss: 0.2768 - accuracy: 0.8802 - val_loss: 0.5384 - val_accuracy: 0.7825
Epoch 5/10
1255/1255 [==============================] - 35s 28ms/step - loss: 0.2252 - accuracy: 0.9054 - val_loss: 0.6585 - val_accuracy: 0.7757
Epoch 6/10
1255/1255 [==============================] - 36s 28ms/step - loss: 0.1815 - accuracy: 0.9246 - val_loss: 0.7902 - val_accuracy: 0.7658
Epoch 7/10
1255/1255 [==============================] - 36s 29ms/step - loss: 0.1473 - accuracy: 0.9393 - val_loss: 0.8573 - val_accura

### Performance Metrics And Accuracy

In [35]:
y_pred1=(model1.predict(X_test) > 0.5).astype("int32")

1236/1236 [==============================] - 7s 5ms/step


In [36]:
y_pred1_jira=(model1.predict(X_final_jira) > 0.5).astype("int32")

26/26 [==============================] - 0s 5ms/step


In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_matrix(y_test,y_pred1)

array([[18074,  4762],
       [ 4662, 12051]], dtype=int64)

In [39]:
confusion_matrix(y_final_jira,y_pred1_jira)

array([[267, 186],
       [122, 253]], dtype=int64)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.7617133176565779

In [41]:
accuracy_score(y_final_jira,y_pred1_jira)

0.6280193236714976

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79     22836
           1       0.72      0.72      0.72     16713

    accuracy                           0.76     39549
   macro avg       0.76      0.76      0.76     39549
weighted avg       0.76      0.76      0.76     39549



In [43]:
print(classification_report(y_final_jira,y_pred1_jira))

              precision    recall  f1-score   support

           0       0.69      0.59      0.63       453
           1       0.58      0.67      0.62       375

    accuracy                           0.63       828
   macro avg       0.63      0.63      0.63       828
weighted avg       0.64      0.63      0.63       828



Injecting some JIRA data into the model

In [49]:
X_train_jira, X_test_jira, y_train_jira, y_test_jira = train_test_split(X_final_jira, y_final_jira, test_size=0.7, random_state=42)

In [50]:
### Finally Training
model1.fit(X_train_jira,y_train_jira,validation_data=(X_test_jira,y_test_jira),epochs=10,batch_size=64)

Epoch 1/10
4/4 [==============================] - 0s 58ms/step - loss: 0.6650 - accuracy: 0.7702 - val_loss: 1.0021 - val_accuracy: 0.6293
Epoch 2/10
4/4 [==============================] - 0s 42ms/step - loss: 0.4264 - accuracy: 0.8266 - val_loss: 0.9552 - val_accuracy: 0.6379
Epoch 3/10
4/4 [==============================] - 0s 47ms/step - loss: 0.2262 - accuracy: 0.9113 - val_loss: 0.9340 - val_accuracy: 0.6483
Epoch 4/10
4/4 [==============================] - 0s 48ms/step - loss: 0.1259 - accuracy: 0.9476 - val_loss: 0.9423 - val_accuracy: 0.6552
Epoch 5/10
4/4 [==============================] - 0s 45ms/step - loss: 0.0773 - accuracy: 0.9798 - val_loss: 0.9640 - val_accuracy: 0.6569
Epoch 6/10
4/4 [==============================] - 0s 45ms/step - loss: 0.0527 - accuracy: 0.9919 - val_loss: 0.9906 - val_accuracy: 0.6586
Epoch 7/10
4/4 [==============================] - 0s 45ms/step - loss: 0.0419 - accuracy: 0.9879 - val_loss: 1.0142 - val_accuracy: 0.6569
Epoch 8/10
4/4 [===========

In [51]:
y_pred1_jira_after_injection=(model1.predict(X_test_jira) > 0.5).astype("int32")

19/19 [==============================] - 0s 4ms/step


In [52]:
confusion_matrix(y_test_jira,y_pred1_jira_after_injection)

array([[236,  87],
       [107, 150]], dtype=int64)

In [53]:
print(classification_report(y_test_jira,y_pred1_jira_after_injection))

              precision    recall  f1-score   support

           0       0.69      0.73      0.71       323
           1       0.63      0.58      0.61       257

    accuracy                           0.67       580
   macro avg       0.66      0.66      0.66       580
weighted avg       0.66      0.67      0.66       580

